In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pprint import pprint


#Spark imports
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


# Start a SparkSession
import findspark
findspark.init()


# Amazon SageMaker and related imports
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer
from sagemaker import get_execution_role
from sagemaker import image_uris
import boto3  # AWS Python sdk

%matplotlib inline

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [21]:
# Import packages
from pyspark.sql import SparkSession
# Import the time module so we can time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [22]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "Resources/heart_attack_prediction_dataset.csv"
#spark.sparkContext.addFile(url)
#df = spark.read.csv(SparkFiles.get("heart_attack_prediction_dataset.csv"), sep=",", header=True)

df = spark.read.csv(url, sep=",", header=True)

df.show()

+----------+---+------+-----------+--------------+----------+--------+--------------+-------+-------+-------------------+-----------------------+---------+-----------------------+--------------+------------+-----------------------+------+------------------+-------------+-------------------------------+-------------------+-------------+-------------+-------------------+-----------------+
|Patient ID|Age|   Sex|Cholesterol|Blood Pressure|Heart Rate|Diabetes|Family History|Smoking|Obesity|Alcohol Consumption|Exercise Hours Per Week|     Diet|Previous Heart Problems|Medication Use|Stress Level|Sedentary Hours Per Day|Income|               BMI|Triglycerides|Physical Activity Days Per Week|Sleep Hours Per Day|      Country|    Continent|         Hemisphere|Heart Attack Risk|
+----------+---+------+-----------+--------------+----------+--------+--------------+-------+-------+-------------------+-----------------------+---------+-----------------------+--------------+------------+-------------

In [23]:
df.printSchema()

root
 |-- Patient ID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Cholesterol: string (nullable = true)
 |-- Blood Pressure: string (nullable = true)
 |-- Heart Rate: string (nullable = true)
 |-- Diabetes: string (nullable = true)
 |-- Family History: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- Obesity: string (nullable = true)
 |-- Alcohol Consumption: string (nullable = true)
 |-- Exercise Hours Per Week: string (nullable = true)
 |-- Diet: string (nullable = true)
 |-- Previous Heart Problems: string (nullable = true)
 |-- Medication Use: string (nullable = true)
 |-- Stress Level: string (nullable = true)
 |-- Sedentary Hours Per Day: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- Triglycerides: string (nullable = true)
 |-- Physical Activity Days Per Week: string (nullable = true)
 |-- Sleep Hours Per Day: string (nullable = true)
 |-- Country: 

In [24]:
df['Diabetes']

Column<'Diabetes'>

In [25]:
type(df['Diabetes'])

pyspark.sql.column.Column

In [26]:
# Convert PySpark DataFrame to Pandas DataFrame for display options
pd_df = df.toPandas()

# Set display options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.precision", 2)

# Display the DataFrame
pd_df.head(10)

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,0,4.168188835442079,Average,0,0,9,6.6150014529140595,261404,31.251232725295402,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,1,1.8132416178634458,Unhealthy,1,0,1,4.963458839757678,285768,27.1949733519874,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,0,2.0783529861178884,Healthy,1,1,9,9.463425838029828,235282,28.176570683909873,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,1,9.82812959348533,Average,1,0,9,7.648980824461007,125640,36.464704293082846,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,0,5.804298820315434,Unhealthy,1,0,6,1.5148209264291386,160555,21.809144180619754,231,1,5,Thailand,Asia,Northern Hemisphere,0
5,ZOO7941,54,Female,297,172/86,48,1,1,1,0,1,0.6250080237057354,Unhealthy,1,1,2,7.798752408582432,241339,20.146839503010046,795,5,10,Germany,Europe,Northern Hemisphere,1
6,WYV0966,90,Male,358,102/73,84,0,0,1,0,1,4.098177090985471,Healthy,0,0,7,0.6273560009569512,190450,28.885810606590454,284,4,10,Canada,North America,Northern Hemisphere,1
7,XXM0972,84,Male,220,131/68,107,0,0,1,1,1,3.427928754300873,Average,0,1,4,10.543780239266841,122093,22.2218617394038,370,6,7,Japan,Asia,Northern Hemisphere,1
8,XCQ5937,20,Male,145,144/105,68,1,0,1,1,0,16.868302239450017,Average,0,0,5,11.348786873498993,25086,35.80990131909641,790,7,4,Brazil,South America,Southern Hemisphere,0
9,FTJ5456,43,Female,248,160/70,55,0,1,1,1,1,0.1945150606299495,Unhealthy,0,0,4,4.055114781794609,209703,22.55891675229818,232,7,7,Japan,Asia,Northern Hemisphere,0


In [27]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Patient ID                       8763 non-null   object
 1   Age                              8763 non-null   object
 2   Sex                              8763 non-null   object
 3   Cholesterol                      8763 non-null   object
 4   Blood Pressure                   8763 non-null   object
 5   Heart Rate                       8763 non-null   object
 6   Diabetes                         8763 non-null   object
 7   Family History                   8763 non-null   object
 8   Smoking                          8763 non-null   object
 9   Obesity                          8763 non-null   object
 10  Alcohol Consumption              8763 non-null   object
 11  Exercise Hours Per Week          8763 non-null   object
 12  Diet                             8

In [28]:
# Split the "Blood Pressure" column into two separate columns for systolic and diastolic pressure
pd_df[['Systolic Pressure', 'Diastolic Pressure']] = pd_df['Blood Pressure'].str.split('/', expand=True)

# Convert the new columns to numeric
pd_df['Systolic Pressure'] = pd.to_numeric(pd_df['Systolic Pressure'])
pd_df['Diastolic Pressure'] = pd.to_numeric(pd_df['Diastolic Pressure'])

# Drop the original "Blood Pressure" column
pd_df.drop('Blood Pressure', axis=1, inplace=True)

pd_df.columns

Index(['Patient ID', 'Age', 'Sex', 'Cholesterol', 'Heart Rate', 'Diabetes',
       'Family History', 'Smoking', 'Obesity', 'Alcohol Consumption',
       'Exercise Hours Per Week', 'Diet', 'Previous Heart Problems',
       'Medication Use', 'Stress Level', 'Sedentary Hours Per Day', 'Income',
       'BMI', 'Triglycerides', 'Physical Activity Days Per Week',
       'Sleep Hours Per Day', 'Country', 'Continent', 'Hemisphere',
       'Heart Attack Risk', 'Systolic Pressure', 'Diastolic Pressure'],
      dtype='object')

In [29]:
# Dictionary mapping values in the 'gender' column to new values
mapping = {'Female':0,'Male':1,
           'Northern Hemisphere':0, 'Southern Hemisphere':1}

# Use the 'map' method to change the values in the 'gender' column
pd_df['Sex'] = pd_df['Sex'].map(mapping)
pd_df['Hemisphere'] = pd_df['Hemisphere'].map(mapping)

# Show DataFrame
pd_df[:5]

,Patient ID,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk,Systolic Pressure,Diastolic Pressure
0,BMW7812,67,1,208,72,0,0,1,0,0,4.168188835442079,Average,0,0,9,6.6150014529140595,261404,31.251232725295402,286,0,6,Argentina,South America,1,0,158,88
1,CZE1114,21,1,389,98,1,1,1,1,1,1.8132416178634458,Unhealthy,1,0,1,4.963458839757678,285768,27.1949733519874,235,1,7,Canada,North America,0,0,165,93
2,BNI9906,21,0,324,72,1,0,0,0,0,2.0783529861178884,Healthy,1,1,9,9.463425838029828,235282,28.176570683909873,587,4,4,France,Europe,0,0,174,99
3,JLN3497,84,1,383,73,1,1,1,0,1,9.82812959348533,Average,1,0,9,7.648980824461007,125640,36.464704293082846,378,3,4,Canada,North America,0,0,163,100
4,GFO8847,66,1,318,93,1,1,1,1,0,5.804298820315434,Unhealthy,1,0,6,1.5148209264291386,160555,21.809144180619754,231,1,5,Thailand,Asia,0,0,91,88


In [51]:
# Check unique values for each column
for col in pd_df.columns:
    print(f"Unique values for column '{col}': {pd_df[col].unique()}")

Unique values for column 'Patient ID': ['BMW7812' 'CZE1114' 'BNI9906' ... 'XKA5925' 'EPE6801' 'ZWN9666']
Unique values for column 'Age': ['67' '21' '84' '66' '54' '90' '20' '43' '73' '71' '77' '60' '88' '69'
 '38' '50' '45' '36' '48' '40' '79' '63' '27' '25' '86' '42' '52' '29'
 '30' '47' '44' '33' '51' '70' '85' '31' '56' '24' '74' '72' '55' '26'
 '53' '46' '57' '22' '35' '39' '80' '65' '83' '82' '28' '19' '75' '18'
 '34' '37' '89' '32' '49' '23' '59' '62' '64' '61' '76' '41' '87' '81'
 '58' '78' '68']
Unique values for column 'Sex': [1 0]
Unique values for column 'Cholesterol': ['208' '389' '324' '383' '318' '297' '358' '220' '145' '248' '373' '374'
 '228' '259' '122' '379' '166' '303' '340' '294' '359' '202' '133' '159'
 '271' '273' '328' '154' '135' '197' '321' '375' '360' '263' '201' '347'
 '129' '229' '251' '121' '190' '185' '279' '336' '192' '180' '203' '368'
 '222' '243' '218' '120' '285' '377' '369' '311' '139' '266' '153' '339'
 '329' '333' '398' '124' '183' '163' '362' '390'

In [52]:
# Define categorical features
categorical_features = ['Country', 'Continent', 'Diet'] 
categorical_dummies = pd.get_dummies(pd_df[categorical_features])
categorical_dummies[:5]

,Country_Argentina,Country_Australia,Country_Brazil,Country_Canada,Country_China,Country_Colombia,Country_France,Country_Germany,Country_India,Country_Italy,Country_Japan,Country_New Zealand,Country_Nigeria,Country_South Africa,Country_South Korea,Country_Spain,Country_Thailand,Country_United Kingdom,Country_United States,Country_Vietnam,Continent_Africa,Continent_Asia,Continent_Australia,Continent_Europe,Continent_North America,Continent_South America,Diet_Average,Diet_Healthy,Diet_Unhealthy
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1


In [53]:
# Define numerical features
numerical_features = [col for col in pd_df.columns if col not in ['Patient ID','Heart Attack Risk'] + categorical_features]

In [54]:
#categorical values
encoded_df = pd.concat([pd_df.drop(categorical_features, axis=1), categorical_dummies], axis=1)
encoded_df['Patient ID'] = pd_df['Patient ID']
encoded_df = encoded_df.set_index('Patient ID')
encoded_df.columns

Index(['Age', 'Sex', 'Cholesterol', 'Heart Rate', 'Diabetes', 'Family History',
       'Smoking', 'Obesity', 'Alcohol Consumption', 'Exercise Hours Per Week',
       'Previous Heart Problems', 'Medication Use', 'Stress Level',
       'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Hemisphere',
       'Heart Attack Risk', 'Systolic Pressure', 'Diastolic Pressure',
       'Country_Argentina', 'Country_Australia', 'Country_Brazil',
       'Country_Canada', 'Country_China', 'Country_Colombia', 'Country_France',
       'Country_Germany', 'Country_India', 'Country_Italy', 'Country_Japan',
       'Country_New Zealand', 'Country_Nigeria', 'Country_South Africa',
       'Country_South Korea', 'Country_Spain', 'Country_Thailand',
       'Country_United Kingdom', 'Country_United States', 'Country_Vietnam',
       'Continent_Africa', 'Continent_Asia', 'Continent_Australia',
       'Continent_Europe', 'Continent_North Am

# Separate features and target variable

In [57]:
y = encoded_df['Heart Attack Risk']
X = encoded_df.drop(['Heart Attack Risk'], axis=1)

In [58]:
X.head(5)

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Hemisphere,Systolic Pressure,Diastolic Pressure,Country_Argentina,Country_Australia,Country_Brazil,Country_Canada,Country_China,Country_Colombia,Country_France,Country_Germany,Country_India,Country_Italy,Country_Japan,Country_New Zealand,Country_Nigeria,Country_South Africa,Country_South Korea,Country_Spain,Country_Thailand,Country_United Kingdom,Country_United States,Country_Vietnam,Continent_Africa,Continent_Asia,Continent_Australia,Continent_Europe,Continent_North America,Continent_South America,Diet_Average,Diet_Healthy,Diet_Unhealthy
Patient ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BMW7812,67,1,208,72,0,0,1,0,0,4.168188835442079,0,0,9,6.6150014529140595,261404,31.251232725295402,286,0,6,1,158,88,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
CZE1114,21,1,389,98,1,1,1,1,1,1.8132416178634458,1,0,1,4.963458839757678,285768,27.1949733519874,235,1,7,0,165,93,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
BNI9906,21,0,324,72,1,0,0,0,0,2.0783529861178884,1,1,9,9.463425838029828,235282,28.176570683909873,587,4,4,0,174,99,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
JLN3497,84,1,383,73,1,1,1,0,1,9.82812959348533,1,0,9,7.648980824461007,125640,36.464704293082846,378,3,4,0,163,100,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
GFO8847,66,1,318,93,1,1,1,1,0,5.804298820315434,1,0,6,1.5148209264291386,160555,21.809144180619754,231,1,5,0,91,88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1


In [59]:
#balance labels y
balance_labels = y.value_counts()
balance_labels


0    5624
1    3139
Name: Heart Attack Risk, dtype: int64

There seems to be a bit of imbalance, with more instances of 
class 0 compared to class 1. Depending on our model's sensitivity to class 
imbalances, we may want to consider techniques like (1) oversampling, 
(2) undersampling, or (3) using class weights during model training to handle this imbalance.

In [60]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train.shape

(7010, 51)

7010: This is the number of samples or instances in your training set. Each row represents a separate observation or data point in your dataset.

26: This is the number of features or variables in your dataset. Each column in your dataset corresponds to a different feature or attribute that is used for training your machine learning model.

So, in summary, you have 7010 training samples, and each sample has 26 features. This information is crucial for understanding the dimensions of your training data, especially when you're working with machine learning models that require input of specific shapes.

In [61]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_model = LogisticRegression(solver="lbfgs",max_iter=200, random_state=1)

In [62]:
# Fit the model using training data
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [64]:
print(f"Training Data Score: {logistic_model.score(X_train, y_train)}")
print(f"Testing Data Score: {logistic_model.score(X_test, y_test)}")

Training Data Score: 0.6393723252496434
Testing Data Score: 0.6514546491728466


In [65]:
# Make a prediction using the testing data
predictions = logistic_model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop = True)
results.head(10)

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,1
4,0,1
5,0,0
6,0,1
7,0,0
8,0,1
9,0,1


In [66]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.6514546491728466

### Evaluation of the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [67]:
from sklearn.metrics import accuracy_score
balanced_accuracy = accuracy_score(y_test, predictions)
balanced_accuracy

0.6514546491728466

In [68]:
    # Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[1142,    0],
       [ 611,    0]])

Let's break down what each of these means:

True Negative (TN): 18673 instances were correctly predicted as class 0 (negative class).

False Positive (FP): 86 instances were incorrectly predicted as class 1 (positive class) when they were actually class 0.

False Negative (FN): 32 instances were incorrectly predicted as class 0 when they were actually class 1.

True Positive (TP): 593 instances were correctly predicted as class 1.

In [69]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.65      1.00      0.79      1142
           1       0.00      0.00      0.00       611

    accuracy                           0.65      1753
   macro avg       0.33      0.50      0.39      1753
weighted avg       0.42      0.65      0.51      1753



/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Precision: Precision is the ratio of correctly predicted positive observations to the total predicted positives. In your case:

Precision for class 0: 0.65 (65% of instances predicted as class 0 were actually class 0)
Precision for class 1: 0.00 (None of the instances predicted as class 1 were actually class 1)
Recall (Sensitivity): Recall is the ratio of correctly predicted positive observations to the all observations in actual class. In your case:

Recall for class 0: 1.00 (All instances of class 0 were correctly predicted)
Recall for class 1: 0.00 (None of the instances of class 1 were correctly predicted)
F1-Score: F1-Score is the weighted average of Precision and Recall. It is a balance between precision and recall. In your case:

F1-Score for class 0: 0.79 (Weighted average of precision and recall for class 0)
F1-Score for class 1: 0.00 (Weighted average of precision and recall for class 1)
Support: The number of actual occurrences of each class in the specified dataset. In your case:

Support for class 0: 1142 instances
Support for class 1: 611 instances
Accuracy: Overall accuracy of the model on the testing data. In your case: 0.65 (65%)

Macro Average: The average of precision, recall, and F1-Score for both classes, without considering class imbalance. In your case:

Macro Average Precision: 0.33
Macro Average Recall: 0.50
Macro Average F1-Score: 0.39
Weighted Average: The average of precision, recall, and F1-Score, weighted by the number of samples in each class. In your case:

Weighted Average Precision: 0.42
Weighted Average Recall: 0.65
Weighted Average F1-Score: 0.51
These metrics provide a more nuanced understanding of your model's performance for each class and overall. In this case, it seems that the model is performing well in predicting class 0 but struggling to predict class 1, as indicated by the low recall and F1-Score for class 1. Depending on the specific goals of your model, you may need to adjust the model or data preprocessing to improve its performance.

In [70]:
#Handling Imbalanced Data:
    # Assuming logistic_model is instantiated
logistic_model = LogisticRegression(class_weight='balanced', random_state=1)

In [72]:
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 13.5 MB/s eta 0:00:00


In [73]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [74]:
# Instantiate a new logistic regression model
logistic_model_resampled = LogisticRegression(random_state=1)

# Fit the model with the resampled data
logistic_model_resampled.fit(X_resampled, y_resampled)

# Make predictions on the testing data
predictions_resampled = logistic_model_resampled.predict(X_test)

In [75]:
#Evaluate the performance
accuracy_resampled = accuracy_score(y_test, predictions_resampled)
classification_report_resampled = classification_report(y_test, predictions_resampled)
confusion_matrix_resampled = confusion_matrix(y_test, predictions_resampled)

print("Accuracy (Resampled):", accuracy_resampled)
print("Classification Report (Resampled):\n", classification_report_resampled)
print("Confusion Matrix (Resampled):\n", confusion_matrix_resampled)

Accuracy (Resampled): 0.4997147746719909
Classification Report (Resampled):
               precision    recall  f1-score   support

           0       0.65      0.49      0.56      1142
           1       0.35      0.51      0.41       611

    accuracy                           0.50      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.55      0.50      0.51      1753

Confusion Matrix (Resampled):
 [[565 577]
 [300 311]]
